<a href="https://colab.research.google.com/github/staaason/Labs_crypto/blob/main/lab4/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Завдання 1** Реалiзуйте тест на простоту Мiллера-Рабiна. Для кожного тестового числа n бi-
нарної довжини 512, або проведiть 100 тестувань, в яких отримується вiдповiдь
“можливо просте”, або отримайте вiдповiдь “складене”. Для кожного тестування
виведiть вiдповiдне випадкове число a.

In [ ]:
import math
import random
import hashlib
import os 


def miller_rabin_test(n, k=100):
    """
    Miller-Rabin primality test to check whether a given integer n is prime.

    :param n: integer to be tested for primality
    :param k: number of iterations for the test
    :return: True if n is probably prime, False otherwise
    """
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    s = 0
    d = n - 1
    while d % 2 == 0:
        s += 1
        d //= 2
    for i in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        print(f'a= {a}\n')
        if x == 1 or x == n - 1:
            continue
        for j in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

for i in range(100):
    n = int('1' + ''.join(random.choice('01') for _ in range(511)), 2)
    print(f'n= {n}:')
    if miller_rabin_test(n):
      print("можливо просте")
    else:
      print("складене")
    print(f'-'* 80)

n= 11002963496127731295016278326726478009135217115939573803649702866183504778478358391812782370616544164045552585365146675045717454031901473339501974787944823:
a= 2786643837288564908333874026266301744440101689091186398862169929278759719697222833836098446331634104076697529013777247004375536960439073670370190333688058

складене
--------------------------------------------------------------------------------
n= 9527204141105574356840624032313486801614279102333773778161558217193093006424447610320084298622931619972063620524353698396478433726427972311020491583468082:
складене
--------------------------------------------------------------------------------
n= 9612589923019411152692107298070852489793531157703110037821563439685195533434929865320567979865035566494645882625169017644223133507400662206947530124993810:
складене
--------------------------------------------------------------------------------
n= 963740775812114796524512881624493864388951084061228105043605659153847629681270835996953180

**Завдання 2** Реалiзуйте криптосистему RSA для секретних модулiв бiнарної довжини 512. Для
дешифрування використайте китайську теорему про остачi.

In [ ]:
class RSA:
    def __init__(self):
        pass

    def modular_inverse(self, e, phi):
        """
        Computes the modular inverse of e mod phi using the extended Euclidean algorithm.

        :param e: An integer representing the value of e in RSA.
        :param phi: An integer representing the value of phi in RSA.
        :return: An integer representing the modular inverse of e mod phi.
        """

        def extended_euclidean_algorithm(a, b):
            """
            A helper function that implements the extended Euclidean algorithm.

            :param a: An integer.
            :param b: An integer.
            :return: A tuple containing the greatest common divisor of a and b, and the Bezout coefficients x and y.
            """

            if b == 0:
                return a, 1, 0
            else:
                gcd, x_, y_ = extended_euclidean_algorithm(b, a % b)
                x = y_
                y = x_ - (a // b) * y_
                return gcd, x, y

        _, d, _ = extended_euclidean_algorithm(e, phi)
        return d % phi

    def generate_large_prime(self, num_bits):
        """
        Generates a large prime number of num_bits bits using a probabilistic primality test.

        :param num_bits: An integer representing the number of bits in the prime.
        :return: A large prime number.
        """
        prime_candidate = random.getrandbits(num_bits) | (1 << num_bits - 1) | 1
        while not miller_rabin_test(prime_candidate):
            prime_candidate += 2
        return prime_candidate

    def newkeys(self, num_bits):
        """
        Generates a new RSA key pair.

        :param num_bits: An integer representing the number of bits in the modulus.
        :return: A tuple containing the public key (e, n) and the private key (d, n, p, q).
        """
        p = self.generate_large_prime(num_bits // 2)
        q = self.generate_large_prime(num_bits // 2)
        while p == q:
            q = self.generate_large_prime(num_bits // 2)

        n = p * q
        phi = (p - 1) * (q - 1)
        # e = 65537
        e = random.randint(2, phi - 1)
        while math.gcd(e, phi) != 1:
            e = random.randint(2, phi - 1)
        d = self.modular_inverse(e, phi)
        return (e, n), (d, n, p, q)

    def encrypt(self, message, public_key):
        """
        Encrypts a message using the RSA public key.

        :param message: An integer representing the message to be encrypted.
        :param public_key: A tuple containing the public key (e, n).
        :return: An integer representing the ciphertext.
        """
        e, n = public_key
        cipher = pow(message, e, n)
        return cipher

    def decrypt(self, c, private_key):
        """
        Decrypts a ciphertext using the RSA private key.

        :param c: An integer representing the ciphertext to be decrypted.
        :param private_key: A tuple containing the private key (d, n, p, q).
        :return: A string representing the decrypted plaintext or an error message if the ciphertext is invalid.
        """
        d, n, p, q = private_key
        if c >= n:
            return "ciphertext representative out of range"
        d_p = d % (p - 1)
        d_q = d % (q - 1)
        q_inv = self.modular_inverse(q, p)
        m_1 = pow(c, d_p, p)
        m_2 = pow(c, d_q, q)
        h = (q_inv * (m_1 - m_2)) % p
        m = (m_2 + h * q) % n
        return m


rsa = RSA()
keys = rsa.newkeys(1024)
public_key = keys[0]
private_key = keys[1]
message = "Hello".encode("utf-8")
message = int.from_bytes(message, 'big')
encrypted_text = rsa.encrypt(message, public_key)
decrypted_text = rsa.decrypt(encrypted_text, private_key)
hex_decrypted_message = hex(decrypted_text)[2:].upper()
bytes_decrypted_message = bytes.fromhex(hex_decrypted_message)
decrypted_message = bytes_decrypted_message.decode('utf-8')
print(decrypted_message)

a= 11520220665712881565575253133639060321287301526702573113909558939623449941842436263306588568387368022878210209690933123360044006195307118568854772493696758

a= 6121782517985613299852295780055637057472989989093515380665478212144180942874887853787760682439993545549417699333584770688762745256428952270977061332593131

a= 8831773598877653406920717260798685532385211442488182147543529484219519668644147528574000658684680673696195129185038164048580310084756849142348143328204665

a= 11923077965851327544045118137588483447759216324644542710569256646609801072831887953778433512459284122192452236339755862689509986392215631348474608436192142

a= 1251932733380624556395654662761364615612262913623018640616704475240081604732358226143548288967571375715231846039666076498305168400900796157553581900149555

a= 1742545607808372725585057713521570610622884090165636402125005189154476525990018575164483908120207527243788453628415235651812059699721012121362192848916451

a= 14154758840828634678329289027116108522848

**Завдання 3** Реалiзуйте RSA-OAEP.

In [ ]:
class RSA_OAEP:
    def __init__(self):
        self.rsa = RSA()

    # from wiki https://en.wikipedia.org/wiki/Mask_generation_function
    def mgf1(self, seed: bytes, length: int, hash_func=hashlib.sha1) -> bytes:
        """
        MGF1 (Mask Generation Function 1) as defined in the PKCS #1 v2.1 standard.

        :param seed: The seed value to use for the mask generation.
        :param length: The desired length of the mask.
        :param hash_func: The hash function to use for the mask generation. Default is hashlib.sha1.
        :return: The mask generated using the specified seed, length, and hash function.
        """
        hLen = hash_func().digest_size
        # 1.If l > 2^32(hLen), output "mask too long" and stop.
        if length > (hLen << 32):
            raise ValueError("mask too long")
        # 2.Let T  be the empty octet string.
        T = b""
        # 3.For counter from 0 to \lceil{l / hLen}\rceil-1, do the following:
        counter = 0
        while len(T) < length:
            # a.Convert counter to an octet string C of length 4 with the primitive I2OSP: C = I2OSP (counter, 4)
            C = int.to_bytes(counter, 4, 'big')
            # b.Concatenate the hash of the seed Z and C to the octet string T: T = T || Hash (Z || C)
            T += hash_func(seed + C).digest()
            counter += 1
        # 4.Output the leading l octets of T as the octet string mask.
        return T[:length]

    def generate_keys(self, num_bits):
        """
        Generates a pair of RSA keys with the specified number of bits using the PyCrypto library.

        :param num_bits: The number of bits to use for the RSA key.
        :return: A tuple containing the public and private keys generated using the specified number of bits.
        """
        return self.rsa.newkeys(num_bits)

    def xor_bytes(self, b1, b2):
        """
        Computes the bitwise XOR of two byte strings of equal length and returns the result.

        :param b1: The first byte string to XOR.
        :param b2: The second byte string to XOR.
        :return: The byte string resulting from XORing b1 and b2.
        """
        if len(b1) != len(b2):
            raise ValueError("Byte strings must be of equal length")
        return bytes([a ^ b for a, b in zip(b1, b2)])

    def OS2IP(self, x):
        """
        OS2IP converts an octet string to a nonnegative integer

        :param x: x octet string to be converted
        :return: x corresponding nonnegative integer
        """
        return int.from_bytes(x, 'big')

    def I2OSP(self, x, xLen):
        """
        Converts a nonnegative integer to an octet string of a specified length.
        :param x: The nonnegative integer to be converted.
        :param xLen: The length of the output octet string.
        :return: The octet string representation of the input integer.
        """
        if x >= 256 ** xLen:
            raise ValueError("Integer too large for specified length")
        octets = []
        for i in range(xLen - 1, -1, -1):
            octet = (x >> (8 * i)) & 0xff
            octets.append(octet)
        return bytes(octets)

    def encrypt(self, public_key, message, hash_func=hashlib.sha1, label=''):
        """
        :param hash_func:  hash_func is the chosen hash function in hashlib library
        :param public_key: recipient's RSA public key (k denotes the length in
               octets of the RSA modulus n)
        :param message: message to be encrypted, an octet string of length mLen,
               where mLen <= k - 2hLen - 2
        :param label: optional label to be associated with the message; the
               default value for Label, if Label is not provided, is the empty
               string
        :return:  ciphertext, an octet string of length k
        """
        e, n = public_key
        hash_object = hash_func()
        if len(label.encode('utf-8')) > (2 ** 61 - 1):
            raise ValueError("label too long")
        k = len(n.to_bytes((n.bit_length() + 7) // 8, 'big'))
        hLen = hash_object.digest_size
        mLen = len(message)
        if mLen > k - 2 * hLen - 2:
            raise ValueError("message too long")
        hash_object.update(label.encode('utf-8'))
        lHash = hash_object.digest()
        pad_len = k - mLen - 2 * hLen - 2
        ps = b'\x00' * pad_len
        db = lHash + ps + b'\x01' + message
        seed = os.urandom(hLen)
        dbMask = self.mgf1(seed, k - hLen - 1)
        maskedDB = self.xor_bytes(db, dbMask)
        seedMask = self.mgf1(maskedDB, hLen)
        maskedSeed = self.xor_bytes(seed, seedMask)
        EM = b'\x00' + maskedSeed + maskedDB
        m = self.OS2IP(EM)
        c = self.rsa.encrypt(m, public_key)
        return self.I2OSP(c, k)

    def decrypt(self, private_key, cipher, label='', hash_func=hashlib.sha1):
        """
        :param hash_func: hash_func is the chosen hash function in hashlib library
        :param private_key: recipient's RSA private key (k denotes the length in
               octets of the RSA modulus n), where k >= 2hLen + 2
        :param cipher: ciphertext to be decrypted, an octet string of length k
        :param label: optional label whose association with the message is to
               be verified; the default value for L, if L is not
               provided, is the empty string
        :return: message, an octet string of length mLen, where
               mLen <= k - 2hLen - 2
        """

        if len(label.encode('utf-8')) > (2 ** 61 - 1):
            raise ValueError("label too long")
        _, n, _, _ = private_key
        hash_object = hash_func()
        k = len(n.to_bytes((n.bit_length() + 7) // 8, 'big'))
        hLen = hash_object.digest_size
        if len(cipher) != k:
            raise ValueError("decryption error")
        if k < 2 * hLen + 2:
            raise ValueError("decryption error")
        c = self.OS2IP(cipher)
        m = self.rsa.decrypt(c, private_key)
        if m == "ciphertext representative out of range":
            raise ValueError("decryption error")
        EM = self.I2OSP(m, k)
        hash_object.update(label.encode('utf-8'))
        lHash = hash_object.digest()
        Y, maskedSeed, maskedDB = EM[0], EM[1:hLen + 1], EM[hLen + 1:]
        if Y != b'\x00' and Y != 0:
            raise ValueError('decryption error')
        seedMask = self.mgf1(maskedDB, hLen)
        seed = self.xor_bytes(maskedSeed, seedMask)
        dbMask = self.mgf1(seed, k - hLen - 1)
        DB = self.xor_bytes(maskedDB, dbMask)
        lHash_ = DB[:hLen]
        if lHash != lHash_:
            raise ValueError("decryption error")
        i = hLen
        while i < len(DB):
            if DB[i] == 0:
                i += 1
            elif DB[i] == 1:
                break
            else:
                raise ValueError("decryption error")
        m = DB[i + 1:]
        return m


RSAOAEP = RSA_OAEP()
keys = RSAOAEP.generate_keys(512)
public_key = keys[0]
private_key = keys[1]
message = "code working".encode('utf-8')
encrypted = RSAOAEP.encrypt(public_key, message)
print(encrypted)
decrypted = RSAOAEP.decrypt(private_key, encrypted)
print(decrypted)


a= 8261791065591349376979659741922844555014971586061410110208406980273612192702

a= 13778230187649044673490329763319650899687734013103657651237673823810448396525

a= 58619950047617128674835701157858860439197129704238624666431578601011826412913

a= 28023279892448671037118465200418793213644193956424877721228486869903021807529

a= 46312405276614530048682136786923977334111644503427643723045256933013889787677

a= 35308730614665360180257603370353290945927839651544760023916934151674393802574

a= 40177195326410564836551790707079789248050510556805050219282228556128331610763

a= 61601611929921404175795679136093342948742627744161108225426127964138926443825

a= 1489180120349842076862630216935603159711259725836271802143304086237107301494

a= 44516324988721630755557762995329171136148758499208783200489438444475686525667

a= 12229233741439258543365983030844925174810771673970176864259764839001323576754

a= 969946542158287932855106928659373924674390433329682841122915127432986145684

a= 77049136911355002